# Wildfire Risk - Ingest - Athena Table Setup
__Team 3 - Dave Friesen, John Chen, and Kyle Dalope__<br>
__ADS-508-02-SP23__<br><br>
__GitHub link: https://github.com/davefriesen/wildfire-risk__

In [6]:
__authors__ = ['Dave Friesen', 'John Chen', 'Kyle Dalope']
__contact__ = ['dfriesen@sandiego.edu', 'johnchen@sandiego.edu', 'kdalope@sandiego.edu']
__date__ = '2023-03-20'
__license__ = 'MIT'
__version__ = '1.0.1'

# Setup Basics

In [7]:
# Import basic libraries
import boto3
import sagemaker

# Import data access libraries
import pandas as pd
# !pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

In [8]:
# Establish session fundamentals
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.Session().client(service_name='sagemaker', region_name=region)

# Register S3 CSVs with Athena

In [9]:
ingest_create_athena_table_passed = False

In [10]:
%store -r ingest_create_athena_db_passed
try:
    ingest_create_athena_db_passed
    print(ingest_create_athena_db_passed)
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS NOTEBOOK *********************************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")
    
if not ingest_create_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")    

True
[OK]


In [11]:
%store -r s3_private_path
try:
    s3_private_path
    print(s3_private_path)
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS NOTEBOOK *********************************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

s3://sagemaker-us-east-1-074876746575/widfire-risk/csv/


In [12]:
# Set Athena database name
database_name = 'dsoaws'

# Set S3 staging directory (temp directory for Athena queries)
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# Establish S3 connection
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

#table_name_tsv = "amazon_reviews_tsv"

# SQL statement to execute
#statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
#         marketplace string,
#         customer_id string,
#         review_id string,
#         product_id string,
#         product_parent string,
#         product_title string,
#         product_category string,
#         star_rating int,
#         helpful_votes int,
#         total_votes int,
#         vine string,
#         verified_purchase string,
#         review_headline string,
#         review_body string,
#         review_date string
#) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION '{}'
#TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
#    database_name, table_name_tsv, s3_private_path_tsv
#)
#print(statement)

#pd.read_sql(statement, conn)

# Verify table creation
#statement = "SHOW TABLES in {}".format(database_name)
#df_show = pd.read_sql(statement, conn)

#if table_name in df_show.values:
#    ingest_create_athena_table_passed = True
#%store ingest_create_athena_table_passed

#df_show.head(5)

# Run A Sample Query

In [13]:
#product_category = "Digital_Software"

#statement = """SELECT * FROM {}.{}
#    WHERE product_category = '{}' LIMIT 100""".format(
#    database_name, table_name_tsv, product_category
#)

#print(statement)

In [14]:
#df = pd.read_sql(statement, conn)
#df.head(5)

In [15]:
#if not df.empty:
#    print("[OK]")
#else:
#    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
#    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
#    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

# Review the New Athena Table in the Glue Catalog

In [16]:
#from IPython.core.display import display, HTML

#display(
#    HTML(
#        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
#            region
#        )
#    )
#)

# Store Variables and Close Session

In [17]:
# Store variables for subsequent notebooks
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path                            -> 's3://sagemaker-us-east-1-074876746575/widfire-ris
s3_public_path                             -> 's3://wildfire-risk/'
setup_dependencies_passed                  -> True
setup_iam_roles_passed                     -> True
setup_instance_check_passed                -> True
setup_s3_bucket_passed                     -> True


In [18]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [12]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>